## SKT AI / KoBART 

https://github.com/SKT-AI/KoBART


In [ ]:
pip install git+https://github.com/SKT-AI/KoBART#egg=kobart

In [ ]:
from transformers import BartModel
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer
kobart_tokenizer = get_kobart_tokenizer()
model = BartModel.from_pretrained(get_pytorch_kobart_model())


In [ ]:
inputs = kobart_tokenizer(['반가워요. 나는 지금 여기에 있어요.'], return_tensors='pt')
context = model(inputs['input_ids'], )
type(context)

In [ ]:
context.encoder_last_hidden_state.shape

In [ ]:
dir(context)

In [ ]:

import sys
import requests
import pandas as pd
import numpy as np
import html
from bs4 import BeautifulSoup
import bs4
import re
import json
import time
from datetime import datetime
import ipywidgets

In [ ]:
#네이버 API 아이디 키 값
client_id = "" #1.에서 취득한 아이디 넣기
client_secret = ""  #1. 에서 취득한 키 넣기

In [ ]:

def GetNaverNews(search_word, num= 100):
    # search_word = '부동산' #검색어
    global client_id, client_secret

    encode_type = 'json'  # 출력 방식 json 또는 xml
    max_display = 100  # 출력 뉴스 수
    sort = 'date'  # 결과값의 정렬기준 시간순 date, 관련도 순 sim

    bFirst = True
    for start in range(1, num, 100):
        url = f"https://openapi.naver.com/v1/search/news.{encode_type}?query={search_word}&display={str(int(max_display))}&start={str(int(start))}&sort={sort}"

        # 헤더에 아이디와 키 정보 넣기
        headers = {'X-Naver-Client-Id': client_id,
                   'X-Naver-Client-Secret': client_secret
                   }

        # HTTP요청 보내기
        r = requests.get(url, headers=headers)
        # 요청 결과 보기 200 이면 정상적으로 요청 완료
        if r.status_code == 200:
            if bFirst:
                data = pd.DataFrame(r.json()['items'])
                bFirst = False
            else:
                data = pd.concat([data, pd.DataFrame(r.json()['items'])], axis=0)

    # 태그 삭제
    data['title'] = data['title'].apply(lambda x: re.sub('(<([^>]+)>)', '', html.unescape(x)))
    data['description'] = data['description'].apply(lambda x: re.sub('(<([^>]+)>)', '', html.unescape(x)))

    return data

In [ ]:
data = GetNaverNews('김포시', num=100)
data.head(10)

In [ ]:
contexts = np.zeros([data.shape[0], context.encoder_last_hidden_state.shape[-1]])
contexts.shape

In [ ]:
for i, row in data.iterrows():
  inputs = kobart_tokenizer([row['description']], return_tensors='pt')
  context = model(inputs['input_ids'], )
  contexts[i, :] = context.encoder_last_hidden_state.detach().numpy()[0,0,:]
  sys.stdout.write('\r[' + str(i).zfill(4) + '/' + str(data.shape[0]).zfill(4) + '] ' + row['title'])

In [ ]:
from sklearn.cluster import KMeans #kmeans
from scipy.spatial.distance import cdist


#kmeans 클러스터링 진행
n_clusters= 10
clt = KMeans(n_clusters=n_clusters)
clt.fit(contexts)

#중심점간 거리를 교차 계산
centroid = clt.cluster_centers_.astype(np.int64())
dist = cdist(centroid,centroid)


centroid = clt.cluster_centers_.astype(np.int64())
centroid.shape


In [ ]:
#%%
#각 기사별로 소속 클러스터로 맵핑한다.
xdata = clt.predict(contexts)
xdata = xdata.reshape(xdata.shape[0], 1)

xdata.shape

In [ ]:

#각 클러스터별 출력한다.

def display_cluster_image(cnum = 0) :
  print(data.loc[np.where(xdata == cnum)[0], 'title'])


ipywidgets.interact(display_cluster_image, cnum=(0, n_clusters, 1) )